In [ ]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
plt.rcParams['image.cmap'] = 'gray' # we want our images to be show black and white, not heat-mapped

With `tf.InteractiveSession` we also create a graph and set it as a default one. This allows us to add stuff to the graph without fancy `with ...default` statements, making our lives easier when doing interactive stuff in Jupyter. 

But in production it's recommended to use the previous method of defining a graph and operations within it.


In [ ]:
session = tf.InteractiveSession()

We use a library function from tensorflow to download and read MNIST so we can train a model on it.

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Data exploration

This is a very important step. Knowing your data, both the low-level details (shapes of the arrays, types of the arrays, domain (are our images in \[0, 255\] or in \[0.0, 1.0\]?), class distribution) and high-level details (what kind of images do we have (printed/handwritten, noisy or clean...), what kind of text do we have (well-formed text from a newspaper, archaic text from a classical novel or tweets with typos)) is **EXTREEMLY IMPORTANT**.

`mnist` is a well structured dataset object with three fields useful to us: `train`, `validation` and `test`, containing different part of the dataset splits. Each of them contains two fields: `images` and `labels`.

Note that IRL importing the data and preprocessing it is **OUR** job and usualy structure is not so clear ;) (we have images with different sizes , we have text with different lengths, etc. but this is out of the scope of this introduction)

In [ ]:
mnist.train.images.shape

So we have 55 000 images 784 by ...? Reading about MNIST, we see that 784 is the flattened size. The real size is 28x28.

In [ ]:
np.min(mnist.train.images[0]), np.max(mnist.train.images[0])

So our images have pixel values between 0 and 1. That's good, because we don't have to rescale them (almost all ML **MUST** receive values in the ranges \[0, 1\] or \[-1, 1\]).

In [ ]:
plt.imshow(mnist.train.images[0])

Oops. We have to reshape the input to "look" like an image.

In [ ]:
plt.imshow(mnist.train.images[0].reshape([28, 28]))

Calling the `reshape` method on a numpy array (or the function `tf.reshape` for tensors) returns a new array (or tensor) with the same values, but interpreted in a different shape.

In [ ]:
mnist.train.labels.shape

We asked for one_hot encoded labels. That means we have a vector of 9 zeros and 1 one. The position of the one encodes the expected label.

In [ ]:
mnist.train.labels[0]

We can exploit that encoding to compute the class distribution in our training dataset. The mean value of the `i`-th column will give us the ratio of the number of samples of class `i` and all samples.

In [ ]:
np.mean(mnist.train.labels, axis=0) # we mean the values across the 0 axis

# Simple MNIST model

Now after we're done exploring our data we can finally start using ML.

In [ ]:
image = tf.placeholder(tf.float32, [None, 784], name='image')
gt_label = tf.placeholder(tf.float32, [None, 10], name='gt_label')

We define the inputs to our model. We'll pass a batch of image into `image` and during training or validation pass a batch of corresponding ground truth labels into `gt_label`. Since we don't want to commit to a batch size we can pass a `None` in that dimention of the placeholder telling TF "accept any value for that dimention".

In [ ]:
kernel = tf.Variable(tf.random_normal([784, 10], stddev=1e-3), name='kernel')
bias = tf.Variable(tf.zeros([10]), name='bias')

We define our model parameters. Since this is just a linear model we need only a kernel and a bias.

In [ ]:
logits = tf.matmul(image, kernel) + bias[tf.newaxis, :]

We define our model computation. There are a number of caveats here. 

First is the `matmul`. We all know matrix multiplication is not cumulative. It's not even defined if we swap the places  of its operands. A good rule of thumb for `{tf,np}.matmul` is that you can multiply arrays/tensors with shapes `[a,b]` by arrays/tensors with shapes `[b,c]` and get shapes `[a,c]` (you must pass the same "middle shape size").

The second one is the addition. The multiplication result has shape `[batch_size, 10]`, but the bias has shape `[10]`. Actually we can add them directly because the bias shape will autobroadcast to `[1, 10]`, because they have different rank (and the rank is padded on the left). After that the shape component is broadcasted from `1` to `batch_size` automatically by TF. What we've done is to explicitly make them of equal rank and then broadcast only the `batch_size` value. Note that not all operations broadcast the shapes! [More info on broadcasting](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html). TF broadcasting is more or less the same as np.

In [ ]:
predicted_label_probs = tf.nn.softmax(logits)
predicted_label = tf.argmax(predicted_label_probs, axis=-1)

We apply `softmax` to get probabilities out of our logits. We also compute the scalar for our predicted label using argmax.

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=gt_label, 
    logits=logits,
)
loss = tf.reduce_mean(loss) # since we want the average loss

We use `softmax_cross_entropy_with_logits_v2` to compute our loss. For some reason TF doesn't have a cross_entropy without an activation function, so we'll have to use that. Also the not `v2` function is deprecated. And the use of keyword args is mandatory.

In [ ]:
global_step = tf.Variable(0, trainable=False, dtype=tf.int32, name='global_step')
optimizer = tf.train.GradientDescentOptimizer(1e-3) # magic value for our learning rate
train_op = optimizer.minimize(loss, global_step=global_step)

`global_step` is a TF variable holding the number of steps we've been training. We mark it as "non-trainable" since we don't have to train it. 

We create an optimizer to optimize our parameters. We'll use the most simple one - the vanilla `GradientDescentOptimizer`. We also pass a learning rate `1e-3`. Setting the "right" learning rate is **VERY** important. Also during one training you probably will have multiple "right" values for the learning rate for different steps during the training. One variant is to stop the training and resume with the new "right" one. There are also a couple of functions where you can decay the learning rate based on `global_step`.

`train_op` is the gradient descent step. It computes the backpropagation pass, then computes the gradient updates and `assign`s new values for each relevant `trainable` variables. Each variable is trainable if said otherwise. It also automatically increments the `global_step` variable (if passed).

In [ ]:
gt_label_index = tf.argmax(gt_label, axis=-1)
gt_match = tf.equal(predicted_label, gt_label_index)
accuracy = tf.reduce_mean(tf.cast(gt_match, tf.float32), axis=0)

In [ ]:
init_op = tf.global_variables_initializer()

In [ ]:
session.run(init_op)

In [ ]:
session.run(predicted_label_probs, {image: mnist.train.images[0:1]})

In [ ]:
# Train for one epoch
batch_size = 10
for batch_start in range(0, mnist.train.images.shape[0], batch_size):
    # Get the corresponding batches for images and labels
    image_batch = mnist.train.images[batch_start:batch_start + batch_size]
    label_batch = mnist.train.labels[batch_start:batch_start + batch_size]
    
    # Execute one step of the model. Note that train_op doesn't have a value, but it **HAS** to be executed
    # in order to train the model
    global_step_value, loss_value, accuracy_value, _ = session.run(
        [global_step, loss, accuracy, train_op],
        {image: image_batch, gt_label: label_batch}
    )
    if global_step_value % 100 == 0:
        # Print the loss and accuracy of the model on the *Trainng* *batch*
        print("{:6}: loss: {}, accuracy:{} ".format(global_step_value, loss_value, accuracy_value))
    if global_step_value % 1000 == 0:
        # Compute the loss and accuracy on the whole *Validation* set. 
        # Note 0: usually the validation set will be too big to compute validations on the whole set
        # Note 1: there is no `train_op` in the tensors we pass to run, so the model doesn't learn 
        #   anything from the validation set (this would be bad)
        global_step_value, loss_value, accuracy_value = session.run(
            [global_step, loss, accuracy],
            {image: mnist.validation.images, gt_label: mnist.validation.labels}
        )
        print("VAL: {:6}: loss: {}, accuracy:{} ".format(global_step_value, loss_value, accuracy_value))


# Performance exploration

This is another **EXTREEMELY** important step of the process. We want to analyze what's causing our model to err. This analysis will tell us what to do next. Possible next steps:

 - train more (this usually decided based on if the validation performance improves with training the model)
 - weight the examples / add more examples of a specific type (if the model fails on examples from that type)
 - preprocess the data
 - add more data 
 - change the model architecture

Let's check out the examples that are causing problems.

In [ ]:
predicted_label_probs_val, gt_match_val = session.run(
    [predicted_label_probs, gt_match],
    {image: mnist.validation.images, gt_label: mnist.validation.labels}
)

In [ ]:
np.sum(~gt_match_val)

We can use a boolean numpy array as a mask to get only the results where the model has issues.

In [ ]:
failed_predicted_label_probs_val = predicted_label_probs_val[~gt_match_val]
failed_images_val = mnist.validation.images[~gt_match_val]
failed_labels_val = mnist.validation.labels[~gt_match_val]

In [ ]:
failed_predicted_label_probs_val[0], failed_labels_val[0]

In [ ]:
plt.imshow(failed_images_val[0].reshape([28, 28]))

We can also view what's the model's idea of each digit by showing it's kernel.

In [ ]:
kernel_values = session.run(kernel)

In [ ]:
kernel_values.shape

In [ ]:
plt.figure(figsize=(20, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(kernel_values[:, i].reshape([28, 28]))